In [25]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
from elasticsearch_dsl.query import Q
from fuzzywuzzy import fuzz
import re

In [26]:
books = [book.lower() for book in open('../files/books.txt').read().split('\n')]

In [27]:
client = Elasticsearch()
s = Search(using=client)

In [34]:
verse_parser = re.compile(r'(\d\s)?([\w\.]+)\s+([\d:,\-\s\;]+)')
def search(term, sort='relevant'):
    match = verse_parser.match(term)
    if match:
        response = s.query('match', verse=term).highlight('verse').execute()
        for hit in response:
            print(hit.verse, hit.meta.highlight.verse)
        return
    book_match = max(list(map(lambda book: fuzz.ratio(term.strip().lower(), book), books)))
    sorted_books = list(reversed(sorted(books, key=lambda book: fuzz.ratio(term.strip().lower(), book))))
    if book_match >= 60:
        book_query = Q('bool', must=[Q('match', book=sorted_books[0])])
        response = s.query(book_query).highlight('book').execute()
        for hit in response:
            print(hit['book_id'])
            print(' '.join(hit.meta.highlight.book).strip() + ' ' +  hit.verse.split()[-1])
        return
    else:
        query_string = Q(
            'match_phrase', text={'query': term, 'slop': 2})
        response = s.query(query_string).highlight('text').execute()
        for hit in response.hits.hits:
            print(hit['_source']['book_id'])
            print(' '.join(hit['highlight']['text']))
        return

In [35]:
search('Love')

41
If you <em>love</em> those who <em>love</em> you, what credit is that to you? For even sinners <em>love</em> those who <em>love</em> them.
45
<em>Love</em> is patient and is kind; <em>love</em> doesn’t envy. <em>Love</em> doesn’t brag, is not proud,
61
He who doesn’t <em>love</em> doesn’t know God, for God is <em>love</em>.
48
Grace be with all those who <em>love</em> our Lord Jesus Christ with incorruptible <em>love</em>.
19
<em>Love</em> and faithfulness keep the king safe. His throne is sustained by <em>love</em>.
57
Let brotherly <em>love</em> continue.
55
that they may train the young wives to <em>love</em> their husbands, to <em>love</em> their children,
44
<em>Love</em> doesn’t harm a neighbor. <em>Love</em> therefore is the fulfillment of the law.
61
There is no fear in <em>love</em>; but perfect <em>love</em> casts out fear, because fear has punishment. He who fears is not made perfect in <em>love</em>.
19
I <em>love</em> those who <em>love</em> me.
